In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.mlab
import scipy.io.wavfile
import scipy
import os
import time
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import sparse
sns.set()

In [2]:
def processAudio(folder, sound_dimension = [128, 128]):
    location = folder + '/'
    elements = os.listdir(location)
    results,labels = [], []
    for i in elements:
        try:
            fs, data = scipy.io.wavfile.read(location+i)
            try:
                data = np.mean(data, axis = 1)
            except:
                pass
            audio = matplotlib.mlab.specgram(data)[0]
            audio = scipy.misc.imresize(audio, sound_dimension)
            audio = audio.astype('float64')
            results.append(sparse.csr_matrix(audio.flatten()))
            labels.append(folder)
        except Exception as e:
            print(e)
            continue
    return sparse.vstack(results), labels

In [3]:
folders = [i for i in os.listdir(os.getcwd())if i.find('.md') < 0 and i.find('.txt') < 0 and i.find('ipynb') < 0 and i.find('LICENSE') < 0 and i.find('_background_noise_') < 0]

In [4]:
output,label = [], []
for i in folders:
    print(i)
    out = processAudio(i)
    output.append(out[0])
    label += out[1]

yes
marvin
off
happy
bed
house
up
six
go
four
nine
left
no
three
wow
sheila
right
on
five
seven
zero
stop
one
down
bird
tree
eight
dog
two
cat


In [5]:
print(len(label))
output = sparse.vstack(output)
output.shape

64721


(64721, 16384)

In [6]:
import lightgbm as lgb
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
labels = np.unique(label)
target = LabelEncoder().fit_transform(label)
train_X, test_X, train_Y, test_Y = train_test_split(output, target, test_size = 0.2)

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
params_lgd = {
    'boosting_type': 'dart',
    'objective': 'multiclass',
    'colsample_bytree': 0.4,
    'subsample': 0.8,
    'learning_rate': 0.1,
    'silent': False,
    'n_estimators': 10000,
    'reg_lambda': 0.0005,
    'device':'gpu'
    }
clf = lgb.LGBMClassifier(**params_lgd)
lasttime = time.time()
clf.fit(train_X,train_Y, eval_set=[(train_X,train_Y), (test_X,test_Y)], 
        eval_metric='logloss', early_stopping_rounds=20, verbose=False)
print('time taken to fit lgb:', time.time()-lasttime, 'seconds ')

/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:642: UserWarning: max_bin keyword has been found in `params` and will be ignored. Please use max_bin argument of the Dataset constructor to pass this parameter.
  'Please use {0} argument of the Dataset constructor to pass this parameter.'.format(key))
/usr/local/lib/python3.5/dist-packages/lightgbm/basic.py:648: LGBMDeprecationWarning: The `max_bin` parameter is deprecated and will be removed in 2.0.12 version. Please use `params` to pass this parameter.
  'Please use `params` to pass this parameter.', LGBMDeprecationWarning)


time taken to fit lgb: 2628.1320009231567 seconds 


In [9]:
predicted = clf.predict(test_X)
print('accuracy validation set: ', np.mean(predicted == test_Y))

# print scores
print(metrics.classification_report(test_Y, predicted, target_names = labels))

accuracy validation set:  0.649594438007
             precision    recall  f1-score   support

        bed       0.56      0.52      0.54       332
       bird       0.84      0.66      0.74       381
        cat       0.64      0.68      0.66       353
        dog       0.61      0.51      0.55       332
       down       0.61      0.55      0.58       476
      eight       0.68      0.73      0.70       470
       five       0.64      0.57      0.60       466
       four       0.67      0.77      0.71       469
         go       0.52      0.54      0.53       478
      happy       0.87      0.84      0.85       368
      house       0.78      0.64      0.70       347
       left       0.59      0.60      0.59       484
     marvin       0.77      0.78      0.78       361
       nine       0.63      0.66      0.64       480
         no       0.61      0.57      0.59       456
        off       0.59      0.67      0.63       456
         on       0.67      0.59      0.63       482
    